In [1]:
!pip install SpeechRecognition
!pip install vaderSentiment
!pip install pydub
!pip install ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [3]:
import torch
import gdown
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import speech_recognition as sr
from pydub import AudioSegment

# Step 1: Convert Audio to Text (same as before)
def convert_audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export("temp.wav", format="wav")

    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)

    try:
        conversation_text = recognizer.recognize_google(audio_data)
        return conversation_text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Step 2: Text Preprocessing (same as before)
def preprocess_text(text):
    sentences = text.split(".")
    return sentences

# Step 3: Deep Learning Sentiment Analysis
def analyze_sentiment(sentences):
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"  # Pre-trained BERT model for sentiment analysis
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    positive, negative, neutral = 0, 0, 0

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=-1)
        sentiment_score = torch.argmax(probs)

        # Mapping sentiment_score to positive, negative, neutral
        if sentiment_score in [4, 3]:  # 4 and 3 represent strong and weak positive respectively
            positive += 1
        elif sentiment_score in [0, 1]:  # 0 and 1 represent strong and weak negative respectively
            negative += 1
        else:  # 2 represents neutral
            neutral += 1

    return positive, negative, neutral

# Step 4: Percentage Calculation (same as before)
def calculate_percentage(positive, negative, neutral):
    total = positive + negative + neutral
    if total == 0:
        return 0, 0, 0
    positive_percentage = (positive / total) * 100
    negative_percentage = (negative / total) * 100
    neutral_percentage = (neutral / total) * 100
    return positive_percentage, negative_percentage, neutral_percentage

# Main function to run all steps for a single audio file
def process_audio_file(audio_file):
    print(f"Processing file: {audio_file}")

    conversation_text = convert_audio_to_text(audio_file)
    if not conversation_text:
        print(f"No text extracted from {audio_file}.")
        return

    sentences = preprocess_text(conversation_text)

    positive, negative, neutral = analyze_sentiment(sentences)

    positive_percentage, negative_percentage, neutral_percentage = calculate_percentage(positive, negative, neutral)

    print(f"File: {audio_file}")
    print(f"Positive Sentiment: {positive_percentage:.2f}%")
    print(f"Negative Sentiment: {negative_percentage:.2f}%")
    print(f"Neutral Sentiment: {neutral_percentage:.2f}%")

    if positive_percentage > negative_percentage and positive_percentage > neutral_percentage:
        print("The call is classified as Positive in terms of customer satisfaction.")
    elif negative_percentage > positive_percentage and negative_percentage > neutral_percentage:
        print("The call is classified as Negative in terms of customer satisfaction.")
    else:
        print("The call is classified as Neutral in terms of customer satisfaction.")
    print("\n")

# Main function to download multiple files and process them
if __name__ == "__main__":
    urls = [
        "https://drive.google.com/uc?id=1lnyyBrJuZ16haL9In-rAoiyeNxQJoFww",
        "https://drive.google.com/uc?id=1siHFN6GdFIFs9oaqou3sZ__UFtaCq7i7",
        "https://drive.google.com/uc?id=191pj9uBliYFx7XVh6e6iGYPLt5e_Asc3",
        "https://drive.google.com/uc?id=1TFrFWZY0Z7IF3cUMAz3BcSHNLYZjwARQ",
        "https://drive.google.com/uc?id=1Q7L8M2z_YFFhyX4FaP2qYQs_21YjDFVB",
        "https://drive.google.com/uc?id=1olCg0iJ_-eylIKJdsxCz3fHhq79JANgE"
    ]

    for i, url in enumerate(urls):
        output = f"AudioCall_{i+1}.mp3"
        gdown.download(url, output, quiet=False)

        # Process each downloaded audio file
        process_audio_file(output)


Downloading...
From: https://drive.google.com/uc?id=1lnyyBrJuZ16haL9In-rAoiyeNxQJoFww
To: /content/AudioCall_1.mp3
100%|██████████| 2.44M/2.44M [00:00<00:00, 191MB/s]


Processing file: AudioCall_1.mp3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

...

Processing file: AudioCall_5.mp3


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


File: AudioCall_5.mp3
Positive Sentiment: 50.00%
Negative Sentiment: 50.00%
Neutral Sentiment: 0.00%
The call is classified as Neutral in terms of customer satisfaction.




Downloading...
From: https://drive.google.com/uc?id=1olCg0iJ_-eylIKJdsxCz3fHhq79JANgE
To: /content/AudioCall_6.mp3
100%|██████████| 1.62M/1.62M [00:00<00:00, 129MB/s]


Processing file: AudioCall_6.mp3
Could not request results from Google Speech Recognition service; recognition request failed: Bad Request
No text extracted from AudioCall_6.mp3.
